In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [2]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)
mp_drawing = mp.solutions.drawing_utils
drwaing_spec = mp_drawing.DrawingSpec(thickness = 1, circle_radius = 1)


In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    
    if not success:
        continue
    landmarkImg = image.copy()
    start = time.time()
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image.flags.writeable = False
    
    results = face_mesh.process(image)
    
    image.flags.writeable = True
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    img_h, img_w, img_c = image.shape
    
    face_3d= []
    
    face_2d = []
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            #mp.solutions.drawing_utils.draw_landmarks(landmarkImg,face_landmarks) 
            
            imgHeight = image.shape[0]
            imgWidth = image.shape[1]
            
            #right lower corner white box
            #image[imgHeight-200 : imgHeight, imgWidth-100:imgWidth] = (255,255,255)
            #boxStart = (imgHeight-200,imgWidth-100:imgWidth)
            #boxEnd = (imgWidth-100,imgWidth)
            #box starts from 
            
            eyes_2d_left = []
            eyes_2d_right = []
            lips = []
            rightEyeIndex = [ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398]
            leftEyeIndex = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246]
            lipsIndex = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78]
            for idx, lm in enumerate(face_landmarks.landmark):                
               
                if idx in leftEyeIndex:
                    eyes_2d_left.append((int(lm.x*img_w), int(lm.y*img_h)))
                    
                if idx in rightEyeIndex:
                    eyes_2d_right.append((int(lm.x*img_w), int(lm.y*img_h)))
                    
                if idx in lipsIndex:
                    lips.append((int(lm.x*img_w), int(lm.y*img_h)))
                    
                if idx==33 or idx==263 or idx==1 or idx==61 or idx==291 or idx==199:
                    if idx==1:
                        nose_2d = (lm.x*img_w, lm.y*img_h)
                        nose_3d = (lm.x*img_w, lm.y*img_h, lm.z * 3000)
                                   
                    x,y = int(lm.x*img_w), int(lm.y*img_h)
                    #print(x,y)        
                    face_2d.append([x,y])
                    face_3d.append([x,y,lm.z])
                                   
            
            face_2d = np.array(face_2d, dtype = np.float32)
            face_3d = np.array(face_3d, dtype = np.float32)
            print(type(face_2d[0][0]))
            focal_length = 1*img_w
                                   
            cam_matrix = np.array([[focal_length,0,img_h/2],
                                  [0,focal_length,img_w/2],
                                   [0,0,1]
                                  ], dtype= np.float32)
            
            dist_matrix = np.zeros((4,1), dtype = np.float32)
                                   
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d,face_2d,cam_matrix,dist_matrix)                     
                                   
            rmat, jac = cv2.Rodrigues(rot_vec)
            
            angles, mtxr, mtxq, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
                                   
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            personLookingDirection = "Person is "
            drowsinessStatus = ""
            if y<-10:
                personLookingDirection += "Looking left"
            elif x<-10:
                personLookingDirection += "Looking Down\n"
                drowsinessStatus += "Driver might be drowsy"
            elif y>10:
                personLookingDirection += "Looking Right"            
            elif x>10:
                personLookingDirection += "Looking Up"
            else:
                personLookingDirection += "looking Forward"
                                   
            nose_3d_projection , jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
                                   
            p1 = int(nose_2d[0]), int(nose_2d[1])
            p2 = int(nose_2d[0] + y*10), int(nose_2d[1] - x*10)
            print(len(eyes_2d_right))
            x,y,w,h = cv2.boundingRect(np.array(eyes_2d_left))
            x1,y1,w1,h1 = cv2.boundingRect(np.array(eyes_2d_right))
            x2,y2,w2,h2 = cv2.boundingRect(np.array(lips))
            
            print(x,y,w,h)
            print(x1,y1,w1,h1)
            
            image = cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
            image = cv2.rectangle(image,(x1,y1),(x1+w1,y1+h1),(0,0,255),2)
            image = cv2.rectangle(image,(x2,y2),(x2+w2,y2+h2),(0,255,0),2)
            
            #cv2.line(image, p1, p2, (0,255,0),3)
            #fps, driverLookingDirection, driverAttentivenessStatus, 3 solid angles, 5 level drwsiness checkpoint           
            
            cv2.putText(image,personLookingDirection,(20,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            if len(drowsinessStatus)!=0:
                cv2.putText(image,drowsinessStatus,(20,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        
        end = time.time()
        
        fps = 1/(end - start+0.0001)
        
        print("fps : ", fps)  
        fps = str(fps)
        cv2.putText(image,"FPS : "+fps,(20,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
    #cv2.imshow("Landmarks ",landmarkImg)
    cv2.imshow("image", image)
    if cv2.waitKey(1) & 0xFF ==27:
        break
                                   
cap.release()
cv2.destroyAllWindows()

<class 'numpy.float32'>
16
243 241 38 10
328 229 27 6
fps :  13.744406563052703
<class 'numpy.float32'>
16
255 238 41 11
345 229 28 6
fps :  49.770772957320574
<class 'numpy.float32'>
16
280 234 38 8
363 229 25 6
fps :  43.56250649858959
<class 'numpy.float32'>
16
307 232 39 9
389 228 26 8
fps :  41.76244648561248
<class 'numpy.float32'>
16
318 232 41 10
407 227 38 11
fps :  38.65809526121275
<class 'numpy.float32'>
16
330 229 35 10
410 227 36 10
fps :  46.48075617188674
<class 'numpy.float32'>
16
330 229 35 10
409 227 37 11
fps :  44.272933963807404
<class 'numpy.float32'>
16
329 228 36 11
409 227 37 11
fps :  47.61902253375002
<class 'numpy.float32'>
16
329 228 35 10
409 226 37 11
fps :  39.53555018804211
<class 'numpy.float32'>
16
328 227 35 11
408 226 37 11
fps :  45.854707760375426
<class 'numpy.float32'>
16
328 227 35 11
408 225 37 11
fps :  45.66400307250958
<class 'numpy.float32'>
16
328 227 35 11
408 225 37 11
fps :  44.80929792932034
<class 'numpy.float32'>
16
328 227 35 11
4